### Introduction to Querying solution data using DuckDB
[DuckDB](https://duckdb.org) DuckDB allows for very fast querying of a multitude of data sources. Parquet, CSV and Sqlite are supported and are heavily used by Energy Exemplar. DuckDb is available for both Windows and Linux and is pre-configured on our cloud agents when running simulations on the cloud. DuckDb performs much faster than than Pandas and does not put a heavy memory burden on workers when executing queries.
DuckDB has a feature rich CLI, and great [Python API](https://duckdb.org/docs/stable/clients/python/overview.html)
More information and examples here: [Why DuckDb](https://duckdb.org/why_duckdb.html)

### Covered in this Example
1. Python libraries, environment variables
2. Creating "default" views to expose parquet data in subfolders
3. Creating additional views to produce customized reports, writing to parquet and csv

### Configure Python requirements and path variables. 

In [ ]:
import os
import time
import duckdb
import datetime

parquet_root_path = "~/SolutionData"
duck_db_path = "./duck.ddb"
mapping_file_path = ".directorymapping.json"

today = datetime.date.today()
date_string = today.strftime("%Y-%m-%d")

### Configure Default views to around parquet data
 - This can also be produced programatically as shown in [configure_duck.py](../../JupyterSamples/PreAndPostTasks/Scripts/configure_duck.py)

In [ ]:
try:
    with duckdb.connect(duck_db_path) as con:
        con.execute(f"CREATE OR REPLACE VIEW aggregatedseries AS SELECT * FROM '{os.path.join(parquet_root_path, 'aggregatedseries', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW attribute AS SELECT * FROM '{os.path.join(parquet_root_path, 'attribute', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW attributedata AS SELECT * FROM '{os.path.join(parquet_root_path, 'attributedata', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW category AS SELECT * FROM '{os.path.join(parquet_root_path, 'category', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW class AS SELECT * FROM '{os.path.join(parquet_root_path, 'class', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW classgroup AS SELECT * FROM '{os.path.join(parquet_root_path, 'classgroup', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW collection AS SELECT * FROM '{os.path.join(parquet_root_path, 'collection', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW config AS SELECT * FROM '{os.path.join(parquet_root_path, 'config', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW customcolumn AS SELECT * FROM '{os.path.join(parquet_root_path, 'customcolumn', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW data AS SELECT * FROM '{os.path.join(parquet_root_path, 'data', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW fullkeyinfo AS SELECT * FROM '{os.path.join(parquet_root_path, 'fullkeyinfo', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW membershipinfo AS SELECT * FROM '{os.path.join(parquet_root_path, 'membershipinfo', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW memoobject AS SELECT * FROM '{os.path.join(parquet_root_path, 'memoobject', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW object AS SELECT * FROM '{os.path.join(parquet_root_path, 'object', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW objectmeta AS SELECT * FROM '{os.path.join(parquet_root_path, 'objectmeta', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW period AS SELECT * FROM '{os.path.join(parquet_root_path, 'period', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW phase AS SELECT * FROM '{os.path.join(parquet_root_path, 'phase', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW property AS SELECT * FROM '{os.path.join(parquet_root_path, 'property', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW sampleweight AS SELECT * FROM '{os.path.join(parquet_root_path, 'sampleweight', '**', '*.parquet')}';")
        con.execute(f"CREATE OR REPLACE VIEW unit AS SELECT * FROM '{os.path.join(parquet_root_path, 'unit', '**', '*.parquet')}';")

    print("All views created")
except Exception as e: 
        print('Task failed with exception:')
        print(e)
finally:
    print("done")

### Configure some views, and start querying data to write to Parquet and CSV examples
- This example is also shown in [query_data.py](../../JupyterSamples/PreAndPostTasks/Scripts/query_data.py)

In [ ]:
def configure_views(con) -> None:
    """Configure all the base views to pull out specific reporting periods, phases and properties - this could be optimized to produce smaller files by limiting date ranges, regions, properties etc."""
    view_command = """CREATE OR REPLACE VIEW regional_generation_capacity AS
	SELECT
		key.*, 
		d.PeriodId, 
		d.Value, 
	FROM fullkeyinfo AS key
    INNER JOIN data d on d.SeriesId = key.SeriesId
	WHERE
        key.PeriodTypeName = 'Interval' AND
		key.PhaseName = 'ST' AND
		key.ParentClassName = 'System' AND
		key.ChildClassName = 'Generator' AND
		key.PropertyName IN ('Generation', 'Available Capacity');"""

    con.execute(view_command)
    
    view_command = """CREATE OR REPLACE VIEW region_aggregate_totals AS
    SELECT
        PhaseName, BandId, PeriodTypeName, 
        ParentObjectCategoryName, ParentObjectName, ChildObjectCategoryName, 
        ParentClassName, CollectionName, ChildClassName, PropertyName, UnitValue, TimesliceName, ModelName, SampleId, SampleName, PeriodId,
        SUM(Value) as TotalValue
    FROM regional_generation_capacity ag 
    GROUP BY 
        PhaseName, BandId, PeriodTypeName, 
        ParentObjectCategoryName, ParentObjectName, ChildObjectCategoryName, 
        ParentClassName, CollectionName, ChildClassName, PropertyName, UnitValue, TimesliceName, ModelName, SampleId, SampleName, PeriodId;"""
    
    con.execute(view_command)
    
    view_command = """CREATE OR REPLACE VIEW reporting_data AS
    SELECT 
        t.PhaseName, t.BandId, t.PeriodTypeName, 
        t.ParentObjectCategoryName, t.ParentObjectName, t.ChildObjectCategoryName, 
        t.ParentClassName, t.CollectionName, t.ChildClassName, t.PropertyName, t.UnitValue, t.TimesliceName, t.ModelName, t.SampleId, t.SampleName, 
        p.StartDate, p.EndDate, current_localtimestamp() as SolutionDate, 
        t.TotalValue
    FROM region_aggregate_totals t
    INNER JOIN Period p on p.PeriodId = t.PeriodId;"""
    
    con.execute(view_command)

def write_data(con, file_path: str, convert_csv: bool = False) -> None:
    """Persists properties to parquet, and optionally to csv"""
    start_time = time.time()
    query_command = f"COPY (select * from reporting_data) TO '{file_path}' (FORMAT parquet, COMPRESSION zstd, ROW_GROUP_SIZE 100_000);"
    con.execute(query_command)
    execution_time = time.time() - start_time
    print(f"{file_path} produced in: {execution_time} seconds")
    
    if convert_csv:
        start_time = time.time()
        csv_path = file_path.replace(".parquet", ".csv")
        convert_command = f"""COPY (select * from '{file_path}') TO '{csv_path}' (DELIMITER ',');"""
        con.execute(convert_command)
        execution_time = time.time() - start_time
        print(f"{csv_path} produced in: {execution_time} seconds")

try:
    output_path = "./"
    with duckdb.connect(duck_db_path) as con:
        start_time = time.time()           
        configure_views(con)
        region_file_path = os.path.join(output_path, f"solution_data.parquet")
        write_data(con, region_file_path, convert_csv = True)

        execution_time = time.time() - start_time
        print(f"Complete execution took: {execution_time} seconds")

except Exception as e: 
    print('Task failed with exception:')
    print(e)
finally:
    print("done")
    